# Autores colombianos por provincias

In [1]:
import ast
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

## Autores

In [2]:
%%time

autores_df = pd.read_csv(
    'autores7.csv',
    header=0,
    converters={
        'GEOLOC_CAPITAL_AUTOR': ast.literal_eval,
        'GEOLOC_CAPITAL_EJEMPLAR': ast.literal_eval
    }
)
autores_df = autores_df.fillna('')

CPU times: user 3 s, sys: 31.4 ms, total: 3.03 s
Wall time: 3.03 s


In [3]:
autores_df.shape

(87200, 9)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR
0,Adivina en qué pais ...,Spain,Martina Badstuber,Germany,2010,Berlin,Madrid,"(52.52000659999999, 13.404954)","(40.4167754, -3.7037902)"
1,¡No quiero hacer pipí en el orinal!,Spain,Roser Rius,Mexico,2009,Mexico City,Madrid,"(19.4326077, -99.133208)","(40.4167754, -3.7037902)"
2,Humo,Spain,Antón Fortes Torres,Spain,2008,Madrid,Madrid,"(40.4167754, -3.7037902)","(40.4167754, -3.7037902)"
3,Galleta para perros,Spain,Helen Cooper,United Kingdom,2008,London,Madrid,"(51.5073509, -0.1277583)","(40.4167754, -3.7037902)"
4,Autobio,Spain,Cyril Pedrosa,France,2009,Paris,Madrid,"(48.856614, 2.3522219)","(40.4167754, -3.7037902)"


## Autores colombianos

In [5]:
autores_colombianos_df = autores_df[autores_df['PAIS_AUTOR'] == 'Colombia']
autores_colombianos_df = autores_colombianos_df.copy()

In [6]:
autores_colombianos_df.shape

(5551, 9)

In [7]:
autores_colombianos_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR
7,En la laguna más profunda,Colombia,Oscar Collazos,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
90,El gato bandido y otros cuentos,Colombia,Rafael Pombo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
101,Cuentos pintados,Colombia,Rafael Pombo,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
120,Aleida : 10 años,Colombia,Vladdo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"
292,Mitos y leyendas indígenas de Colombia,Colombia,Javier Ocampo López,Colombia,2013,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)"


## Ciudades y departamentos autores

In [8]:
def get_ciudad_departamento_autor_data_wkd(name, lang):
    template = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?authorLabel ?pobLabel ?depLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author wdt:P31 wd:Q5 .
            OPTIONAL {{
                ?author wdt:P106 wd:Q36180 .
            }}
            ?author wdt:P19 ?pob .
            ?pob wdt:P31 wd:Q2555896 .
            ?pob rdfs:label ?pobLabel .
            FILTER(lang(?pobLabel)="{1}")
            OPTIONAL {{
                ?pob wdt:P131 ?dep .
                ?dep wdt:P31 wd:Q215655 .
                ?dep rdfs:label ?depLabel .
                FILTER(lang(?depLabel)="{1}")
            }}
        }}
    """
    query = template.format(name, lang)
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    data = requests.get(url, params={'query': query, 'format': 'json'}).json()
    return data

In [9]:
sparql_dbp = SPARQLWrapper("http://dbpedia.org/sparql")

def get_ciudad_departamento_autor_data_dbp(name, lang):
    template = """
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX yago: <http://dbpedia.org/class/yago/>

        SELECT ?authorLabel ?pobLabel ?depLabel
        WHERE {{
            BIND("{0}"@{1} AS ?authorLabel) .
            ?author rdfs:label ?authorLabel .
            ?author rdf:type dbo:Person .
            OPTIONAL {{
                ?author rdf:type dbo:Writer .
            }}
            ?author dbp:birthPlace ?pob .
            ?pob rdf:type yago:WikicatMunicipalitiesOfColombia .
            ?pob rdfs:label ?pobLabel .
            FILTER(lang(?pobLabel)="{1}")
            OPTIONAL {{
                ?pob dbo:isPartOf ?dep .
                ?dep rdf:type yago:WikicatDepartmentsOfColombia .
                ?dep rdfs:label ?depLabel .
                FILTER(lang(?depLabel)="{1}")
            }}
        }}
    """
    query = template.format(name, lang)
    sparql_dbp.setQuery(query)
    sparql_dbp.setReturnFormat(JSON)
    data = sparql_dbp.query().convert()
    return data

In [10]:
error_autores = set()

def get_ciudad_departamento_autor_data(db, name, lang):
    try:
        if db == 'wdt':
            data = get_ciudad_departamento_autor_data_wkd(name, lang)
        elif db == 'dbp':
            data = get_ciudad_departamento_autor_data_dbp(name, lang)

        pobs = set(d['pobLabel']['value'] for d in data['results']['bindings'])
        if len(pobs) == 0:
            pob = 'UNKNOWN'
        elif len(pobs) == 1:
            pob = pobs.pop()
        else:
            print('MáS DE UNA POBLACIóN PARA', name, ':', pobs)
            pob = pobs.pop()

        deps = set(d['depLabel']['value'] for d in data['results']['bindings'])
        if len(deps) == 0:
            dep = 'UNKNOWN'
        elif len(deps) == 1:
            dep = deps.pop()
        else:
            print('MáS DE UN DEPARTAMENTO PARA', name, ':', deps)
            dep = deps.pop()
    except:
        print('ERROR EN SPARQL PARA', name)
        pob = 'UNKNOWN'
        dep = 'UNKNOWN'
        error_autores.add(name)
    return (pob, dep)

In [11]:
autores_encontrados_list = []

def get_ciudad_departamento_autor(name):
    result = ('DESCONOCIDO', 'DESCONOCIDO')
    for lang in ['es', 'en']:
        for db in ['wdt', 'dbp']:
            r = get_ciudad_departamento_autor_data(db, name, lang)
            if r[0] != 'UNKNOWN':
                autores_encontrados_list.append(name)
                if r[1] == 'UNKNOWN':
                    result = (r[0], 'DESCONOCIDO')
                else: #r[1] != 'UNKNOWN'
                    result = r
                    return result
    return result

In [12]:
%%time

autores_ciudades_departamentos_dict = {}

for x in autores_colombianos_df['AUTOR'].unique():
    autores_ciudades_departamentos_dict[x] = get_ciudad_departamento_autor(x)

ERROR EN SPARQL PARA Javier Ocampo López
ERROR EN SPARQL PARA Javier Ocampo López
ERROR EN SPARQL PARA Pablo Montoya
ERROR EN SPARQL PARA Pablo Montoya
ERROR EN SPARQL PARA Pirry
ERROR EN SPARQL PARA Pirry
ERROR EN SPARQL PARA Octavio Escobar Giraldo
ERROR EN SPARQL PARA Octavio Escobar Giraldo
ERROR EN SPARQL PARA Arturo Escobar
ERROR EN SPARQL PARA Adalberto Agudelo Duque
ERROR EN SPARQL PARA Adalberto Agudelo Duque
ERROR EN SPARQL PARA Evelio Ramírez Martínez
ERROR EN SPARQL PARA Evelio Ramírez Martínez
ERROR EN SPARQL PARA Alfredo Gómez Jaime
ERROR EN SPARQL PARA Alfredo Gómez Jaime
ERROR EN SPARQL PARA Enrique Santos Montejo
ERROR EN SPARQL PARA Enrique Santos Montejo
ERROR EN SPARQL PARA Maruja Vieira
ERROR EN SPARQL PARA Maruja Vieira
ERROR EN SPARQL PARA Alfonso López Pumarejo
ERROR EN SPARQL PARA Alfonso López Pumarejo
ERROR EN SPARQL PARA David Manzur
ERROR EN SPARQL PARA David Manzur
ERROR EN SPARQL PARA Gilberto Alzate Avendaño
ERROR EN SPARQL PARA Gilberto Alzate Avendaño


In [13]:
%%time

for x in error_autores:
    autores_ciudades_departamentos_dict[x] = get_ciudad_departamento_autor(x)

ERROR EN SPARQL PARA Adalberto Agudelo Duque
ERROR EN SPARQL PARA Adalberto Agudelo Duque
ERROR EN SPARQL PARA Octavio Escobar Giraldo
ERROR EN SPARQL PARA Octavio Escobar Giraldo
ERROR EN SPARQL PARA Evelio Ramírez Martínez
ERROR EN SPARQL PARA Evelio Ramírez Martínez
ERROR EN SPARQL PARA Maruja Vieira
ERROR EN SPARQL PARA Maruja Vieira
ERROR EN SPARQL PARA Alfredo Gómez Jaime
ERROR EN SPARQL PARA Alfredo Gómez Jaime
ERROR EN SPARQL PARA Pablo Montoya
ERROR EN SPARQL PARA Pablo Montoya
ERROR EN SPARQL PARA Arturo Robledo Ocampo
ERROR EN SPARQL PARA Arturo Robledo Ocampo
ERROR EN SPARQL PARA César Grajales
ERROR EN SPARQL PARA César Grajales
ERROR EN SPARQL PARA Gustavo Rojas Pinilla
ERROR EN SPARQL PARA Arturo Escobar
ERROR EN SPARQL PARA Pirry
ERROR EN SPARQL PARA Pirry
ERROR EN SPARQL PARA Gilberto Alzate Avendaño
ERROR EN SPARQL PARA Gilberto Alzate Avendaño
ERROR EN SPARQL PARA Enrique Santos Montejo
ERROR EN SPARQL PARA Enrique Santos Montejo
ERROR EN SPARQL PARA Carlos Quintero


In [14]:
%%time

def process_ciudad(x):
    return autores_ciudades_departamentos_dict[x['AUTOR']][0]

autores_colombianos_df['CIUDAD_AUTOR'] = autores_colombianos_df.apply(process_ciudad, axis=1)

CPU times: user 167 ms, sys: 87 µs, total: 167 ms
Wall time: 165 ms


In [15]:
%%time

def process_departamento(x):
    return autores_ciudades_departamentos_dict[x['AUTOR']][1]

autores_colombianos_df['DEPARTAMENTO_AUTOR'] = autores_colombianos_df.apply(process_departamento, axis=1)

CPU times: user 115 ms, sys: 0 ns, total: 115 ms
Wall time: 113 ms


In [16]:
%%time

def process_encontrado(x):
    return x['AUTOR'] in autores_encontrados_list

autores_colombianos_df['ENCONTRADO'] = autores_colombianos_df.apply(process_encontrado, axis=1)

CPU times: user 154 ms, sys: 3.93 ms, total: 158 ms
Wall time: 156 ms


# Saving results

In [17]:
autores_colombianos_df.to_csv('autores_colombianos_1.csv', index=False)